<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_4/02_fine_tuning_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ファインチューニングを使った文章の分類
ファインチューニングを活用し、文章を分類します。

## ライブラリのインストール
ライブラリTransformers、およびnlpをインストールします。  
https://huggingface.co/transformers/index.html

In [ ]:
!pip install transformers
!pip install nlp

## モデルとTokenizerの読み込み
事前学習済みのモデルと、これと紐づいたTokenizerを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
sc_model.cuda()
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## データセットの読み込み
ライブラリnlpを使用して、インターネット・ムービー・データベース (IMDb)を読み込みます。  
読み込んだデータはトークナイザーで処理し、形式を整えます。

In [ ]:
from nlp import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_data, test_data = load_dataset("imdb", split=["train", "test"])

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

test_data = test_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義します。  


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,
    weight_decay = 0.01,
    evaluate_during_training = True,
    logging_dir = "./logs",
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics  = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data
)

## モデルの訓練
設定に基づきモデルを訓練します。


In [ ]:
trainer.train()

## モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価します。

In [ ]:
trainer.evaluate()

## TensorBoardによる結果の表示
TensorBoardを使って、logsフォルダに格納された学習過程を表示します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs